In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
shops = pd.read_csv('shops_eng.csv')
items = pd.read_csv('items_eng.csv')
categories = pd.read_csv('categories_eng.csv')
cot = train


In [3]:
test.tail()

,ID,shop_id,item_id
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648
214199,214199,45,969


In [4]:
train.query('shop_id == 45  & item_id == 15757')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
114917,25.01.2013,0,45,15757,199.0,1.0
1114476,17.11.2013,10,45,15757,199.0,1.0
1114477,08.11.2013,10,45,15757,199.0,1.0
1602131,07.04.2014,15,45,15757,199.0,1.0
1703030,11.05.2014,16,45,15757,199.0,1.0
2358132,17.01.2015,24,45,15757,199.0,1.0
2415097,22.02.2015,25,45,15757,199.0,1.0


In [5]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
shops.head()

,shop_name,shop_id
0,"! Yakutsk Ordzhonikidze, 56 francs",0
1,"! Yakutsk TC ""Central"" fran",1
2,"Adygea TC ""Mega""",2
3,"Balashikha TC ""Oktyabr-Kinomir""",3
4,"Volga TC ""Volga Mall""",4


In [7]:
items.head()

,item_id,category_id,item_name
0,0,40,!! IN THE POWER OF HAPPINESS (PLAST) D
1,1,76,! ABBYY FineReader 12 Professional Edition Ful...
2,2,40,*** IN THE GLORY OF THE GLORY (UNV) D
3,3,40,*** BLUE WAVE (Univ) D
4,4,40,*** BOX (GLASS) D


In [8]:
categories.head()

,category_name,category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


Categories

In [9]:
categories['group_name'] = categories['category_name'].str.extract(r'(^[\w\s]*)')
categories['group_name'] = categories['group_name'].str.strip()
categories['group_id']  = LabelEncoder().fit_transform(categories.group_name.values)
categories.head()

,category_name,category_id,group_name,group_id
0,PC - Headsets / Headphones,0,PC,12
1,Accessories - PS2,1,Accessories,0
2,Accessories - PS3,2,Accessories,0
3,Accessories - PS4,3,Accessories,0
4,Accessories - PSP,4,Accessories,0


Items

In [10]:
items['item_name'] = items['item_name'].str.lower()
items['item_name'] = items['item_name'].str.replace('.', '')
for i in [r'[^\w\d\s\.]', r'\bthe\b', r'\bin\b', r'\bis\b', r'\bfor\b', r'\bof\b', r'\bon\b', r'\band\b',  r'\bto\b', r'\bwith\b' , r'\byo\b']:
    items['item_name'] = items['item_name'].str.replace(i, ' ')
items['item_name'] = items['item_name'].str.replace(r'\b.\b', ' ')
merged_df = items.merge(categories, on = 'category_id')

In [11]:
merged_df.head()

,item_id,category_id,item_name,category_name,group_name,group_id
0,0,40,power happiness plast,Cinema - DVD,Cinema,5
1,2,40,glory glory unv,Cinema - DVD,Cinema,5
2,3,40,blue wave univ,Cinema - DVD,Cinema,5
3,4,40,box glass,Cinema - DVD,Cinema,5
4,5,40,new american graphics uni,Cinema - DVD,Cinema,5


Train Data

In [12]:
train = train.query('0 < item_price < 5000 and 0 < item_cnt_day < 1001')
train = train.replace({'shop_id' : {0:57, 1:58, 11: 10}})
train = train[train['shop_id'].isin(test.shop_id.unique())]
train['date'] = pd.to_datetime(train.date,format='%d.%m.%Y')
train['total_sales'] = train['item_cnt_day']*train['item_price']

In [13]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_sales
0,2013-01-02,0,59,22154,999.00,1.0,999.00
1,2013-01-03,0,25,2552,899.00,1.0,899.00
3,2013-01-06,0,25,2554,1709.05,1.0,1709.05
4,2013-01-15,0,25,2555,1099.00,1.0,1099.00
5,2013-01-10,0,25,2564,349.00,1.0,349.00


In [14]:
grouped_by_month = train.groupby(['date_block_num', 'shop_id', 'item_id']).sum().reset_index().rename(columns = {'item_cnt_day' : 'total_item_cnt'})
grouped_by_month.head()

,date_block_num,shop_id,item_id,item_price,total_item_cnt,total_sales
0,0,2,27,2499.0,1.0,2499.0
1,0,2,33,499.0,1.0,499.0
2,0,2,317,299.0,1.0,299.0
3,0,2,438,299.0,1.0,299.0
4,0,2,471,798.0,2.0,798.0


In [15]:
combined = grouped_by_month.merge(merged_df, on = 'item_id')
combined.sample(5)

,date_block_num,shop_id,item_id,item_price,total_item_cnt,total_sales,category_id,item_name,category_name,group_name,group_id
804559,17,41,7956,4490.00,1.0,4490.0,6,xbox 360 kinect sensor xbox 360 cs el hu pl ru...,Accessories - XBOX 360,Accessories,0
237475,1,15,6659,2997.00,3.0,2997.0,19,sonic generations special edition 3d suppor...,Games - PS3,Games,8
1122281,23,47,1485,9529.28,10.0,11911.6,19,assassin creed iv black flag ps3 russian v...,Games - PS3,Games,8
546824,21,58,12970,199.00,1.0,199.0,55,circle michael duets,Music - CD of local production,Music,11
222883,1,18,3903,299.00,1.0,299.0,55,divo wicked game,Music - CD of local production,Music,11
